In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading CIFAR10 Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Reducing Target innecessary dimension
y_train = np.squeeze(y_train)
y_test  = np.squeeze(y_test)
# Creating Validation subset
x_valid = x_train[40000:]  
y_valid = y_train[40000:]  

x_train = x_train[:40000]
y_train = y_train[:40000]

## Creating DataSet Objects

In [ ]:
# Training
#-----------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])

# Normalize Images
def normalize_img(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

# 1-hot encoding
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth=10)

# Resizing 
def process_images(image, label):
    # Resize images from 32x32 to 224x224
    image = tf.image.resize(image, (224,224))
    return image, label


train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(normalize_img)
train_dataset = train_dataset.map(process_images)

# Divide in batches
bs = 32
train_dataset = train_dataset.batch(bs)
# Repeat
train_dataset = train_dataset.repeat()

In [ ]:
#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Normalize images
valid_dataset = valid_dataset.map(normalize_img)
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Resize
valid_dataset = valid_dataset.map(process_images)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(normalize_img)
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(process_images)
test_dataset = test_dataset.batch(1)

## Quantization layer

In [ ]:
def Quantization_layer(tensor, Quantization = True,signed = True, word_size = 12, frac_size = 6):
    
    factor = 2.0**frac_size
    
    # Quantized max and min values, in case of the need to implement overflow cases.
    #if signed:
    #    Max_Qvalue = ((1 << (word_size-1)) - 1)/factor
    #    Min_Qvalue = -Max_Qvalue - 1
    #else:
    #    Max_Qvalue = ((1 << (word_size)) - 1)/factor
    #    Min_Qvalue = 0
    
    if Quantization:
        return tf.round(tensor*factor) / factor             #Quantization, assuming no overflow
    else:
        return tensor          

## Creating VGG Model

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Lambda


def build_model(input_layer, Quantization = True, signed = True, word_size = 12, frac_size = 6 ):
    
    Arguments = {'Quantization':Quantization, 'signed':signed, 'word_size':word_size, 'frac_size':frac_size}
    QInp      = Lambda(Quantization_layer, arguments = Arguments )(input_layer)
    
    #Conv Block
    Conv1   = Conv2D(filters=64,kernel_size=(3,3),padding="same")(QInp)
    QConv1  = Lambda(Quantization_layer, arguments = Arguments )(Conv1)
    Relu1   = tf.keras.activations.relu(QConv1)
    QRelu1  = Lambda(Quantization_layer, arguments = Arguments )(Relu1)
    
    Conv2   = Conv2D(filters=64,kernel_size=(3,3),padding="same")(QRelu1)
    QConv2  = Lambda(Quantization_layer, arguments = Arguments )(Conv2)
    Relu2   = tf.keras.activations.relu(QConv2)
    QRelu2  = Lambda(Quantization_layer, arguments = Arguments )(Relu2)
    MP2     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu2)
    
    Conv3   = Conv2D(filters=128, kernel_size=(3,3), padding="same")(MP2)
    QConv3  = Lambda(Quantization_layer, arguments = Arguments )(Conv3)
    Relu3   = tf.keras.activations.relu(QConv3)
    QRelu3  = Lambda(Quantization_layer, arguments = Arguments )(Relu3)
    
    Conv4   = Conv2D(filters=128, kernel_size=(3,3), padding="same")(QRelu3)
    QConv4  = Lambda(Quantization_layer, arguments = Arguments )(Conv4)
    Relu4   = tf.keras.activations.relu(QConv4)
    QRelu4  = Lambda(Quantization_layer, arguments = Arguments )(Relu4)
    MP4     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu4)
    
    Conv5   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(MP4)
    QConv5  = Lambda(Quantization_layer, arguments = Arguments )(Conv5)
    Relu5   = tf.keras.activations.relu(QConv5)
    QRelu5  = Lambda(Quantization_layer, arguments = Arguments )(Relu5)
    
    Conv6   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(QRelu5)
    QConv6  = Lambda(Quantization_layer, arguments = Arguments )(Conv6)
    Relu6   = tf.keras.activations.relu(QConv6)
    QRelu6  = Lambda(Quantization_layer, arguments = Arguments )(Relu6)
    
    Conv7   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(QRelu6)
    QConv7  = Lambda(Quantization_layer, arguments = Arguments )(Conv7)
    Relu7   = tf.keras.activations.relu(QConv7)
    QRelu7  = Lambda(Quantization_layer, arguments = Arguments )(Relu7)
    MP7     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu7)
    
    Conv8   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(MP7)
    QConv8  = Lambda(Quantization_layer, arguments = Arguments )(Conv8)
    Relu8   = tf.keras.activations.relu(QConv8)
    QRelu8  = Lambda(Quantization_layer, arguments = Arguments )(Relu8)
    
    Conv9   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu8)
    QConv9  = Lambda(Quantization_layer, arguments = Arguments )(Conv9)
    Relu9   = tf.keras.activations.relu(QConv9)
    QRelu9  = Lambda(Quantization_layer, arguments = Arguments )(Relu9)
    
    Conv10   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu9)
    QConv10  = Lambda(Quantization_layer, arguments = Arguments )(Conv10)
    Relu10   = tf.keras.activations.relu(QConv10)
    QRelu10  = Lambda(Quantization_layer, arguments = Arguments )(Relu10)
    MP10     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu10)
    
    Conv11   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(MP10)
    QConv11  = Lambda(Quantization_layer, arguments = Arguments )(Conv11)
    Relu11   = tf.keras.activations.relu(QConv11)
    QRelu11  = Lambda(Quantization_layer, arguments = Arguments )(Relu11)
    
    Conv12   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu11)
    QConv12  = Lambda(Quantization_layer, arguments = Arguments )(Conv12)
    Relu12   = tf.keras.activations.relu(QConv12)
    QRelu12  = Lambda(Quantization_layer, arguments = Arguments )(Relu12)
    
    Conv13   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu12)
    QConv13  = Lambda(Quantization_layer, arguments = Arguments )(Conv13)
    Relu13   = tf.keras.activations.relu(QConv13)
    QRelu13  = Lambda(Quantization_layer, arguments = Arguments )(Relu13)
    MP13     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu13)
    
    Flat    = Flatten()(MP13)
    
    Dense14  = Dense(4096)(Flat)
    QDense14 = Lambda(Quantization_layer, arguments = Arguments )(Dense14)
    Relu14   = tf.keras.activations.relu(QDense14)
    QRelu14  = Lambda(Quantization_layer, arguments = Arguments )(Relu14)
    
    Dense15  = Dense(4096)(QRelu14)
    QDense15 = Lambda(Quantization_layer, arguments = Arguments )(Dense15)
    Relu15   = tf.keras.activations.relu(QDense15)
    QRelu15  = Lambda(Quantization_layer, arguments = Arguments )(Relu15)
    
    Dense16  = Dense(10)(QRelu15)
    QDense16 = Lambda(Quantization_layer, arguments = Arguments )(Dense16)
    SM16     = tf.keras.activations.softmax(QDense16)
    QSM16    = Lambda(Quantization_layer, arguments = Arguments )(SM16)
    
    return QSM16

In [11]:

input_layer   = tf.keras.Input((224,224,3))
output_layer  = build_model(input_layer, Quantization = False)

VGG16  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
VGG16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
lambda_33 (Lambda)           (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
lambda_34 (Lambda)           (None, 224, 224, 64)      0         
_________________________________________________________________
tf_op_layer_Relu_15 (TensorF [(None, 224, 224, 64)]    0         
_________________________________________________________________
lambda_35 (Lambda)           (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928 

## Training Options

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
VGG16.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Callbacks

In [ ]:
import os
from datetime import datetime


# Save Directory (Change for your own directory)
cwd = os.getcwd()

# Creating SubDirectory
exps_dir = os.path.join(cwd, 'GraphData')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_name = "VGG16"

exp_dir = os.path.join(exps_dir, exp_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
Checkpoints = False
if Checkpoints:
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
    callbacks.append(ckpt_callback)

# ----------------

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1,write_graph=True)  
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks.append(es_callback)

# ---------------------------------

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR c--port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Training

In [15]:
VGG16.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

Train for 1250 steps, validate for 313 steps
Epoch 1/100
1250/1250 [==============================] - 631s 505ms/step - loss: 1.7190 - accuracy: 0.3665 - val_loss: 1.3131 - val_accuracy: 0.5243
Epoch 2/100
1250/1250 [==============================] - 635s 508ms/step - loss: 1.1137 - accuracy: 0.6019 - val_loss: 0.9505 - val_accuracy: 0.6674
Epoch 3/100
1250/1250 [==============================] - 611s 488ms/step - loss: 0.7453 - accuracy: 0.7405 - val_loss: 0.8086 - val_accuracy: 0.7156
Epoch 4/100
1250/1250 [==============================] - 606s 485ms/step - loss: 0.4460 - accuracy: 0.8464 - val_loss: 0.7763 - val_accuracy: 0.7461
Epoch 5/100
1250/1250 [==============================] - 606s 485ms/step - loss: 0.1842 - accuracy: 0.9385 - val_loss: 1.1628 - val_accuracy: 0.7399
Epoch 6/100
1250/1250 [==============================] - 606s 485ms/step - loss: 0.0878 - accuracy: 0.9707 - val_loss: 1.0038 - val_accuracy: 0.7626
Epoch 7/100
1250/1250 [==============================] - 605s

## Saving Weights

In [ ]:
Wgt_dir = os.path.join(cwd, 'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir, 'Weights')
VGG16.save_weights(Wgt_dir)